In [1]:
# ========================
# 1. CREACIÓN DISPONIBILIDAD DOCENTE SIMULADA
# ========================

import pandas as pd
import numpy as np
import random

In [2]:

# Cargar archivos necesarios
docentes_minimos = pd.read_csv("..\\Data\\Extraidos\\docentes_minimos.csv", sep=";", encoding="utf-8-sig")
duracion_asignatura = pd.read_csv("..\\Data\\Extraidos\\duracion_asignatura.csv", sep=";", encoding="utf-8-sig")
# Crear diccionarios para duración y ciclo
ciclo_dict = duracion_asignatura.drop_duplicates(subset="CodAsignatura").set_index("CodAsignatura")["Ciclo"].to_dict()

# Inicializar parámetros
np.random.seed(32)
contador_docente = 0
filas_disponibilidad = []
excepciones = 3  # generar 3 horarios sin reglas

for _, row in docentes_minimos.iterrows():
    cod_asig = row["CodAsignatura"]
    duracion = float(row["DuracionHoras"])
    ciclo = int(ciclo_dict.get(cod_asig, 1))  # default ciclo 1 si no se encuentra

    # Validar cantidad mínima de docentes
    try:
        min_docentes = int(row["NumGrupos"])
    except:
        min_docentes = 1

    if min_docentes < 1:
        min_docentes = 1

    for _ in range(min_docentes):
        cod_docente = f"Z{str(contador_docente).zfill(6)}"
        contador_docente += 1

        # Reglas según ciclo
        if excepciones > 0:
            posibles_dias = ["LUNES", "MARTES", "MIERCOLES", "JUEVES", "VIERNES", "SABADO"]
            horas_validas = pd.date_range("08:00", "21:00", freq="1h").strftime("%H:%M").tolist()
            excepciones -= 1
        elif ciclo >= 6:
            dia = np.random.choice(["LUNES", "MARTES", "MIERCOLES", "JUEVES", "VIERNES", "SABADO"])
            if dia == "SABADO":
                horas_validas = pd.date_range("08:00", "21:00", freq="1h").strftime("%H:%M").tolist()
            else:
                horas_validas = pd.date_range("18:00", "21:00", freq="1h").strftime("%H:%M").tolist()
        else:
            dia = np.random.choice(["LUNES", "MARTES", "MIERCOLES", "JUEVES", "VIERNES"])
            horas_validas = pd.date_range("08:00", "12:00", freq="1h").strftime("%H:%M").tolist()
            horas_validas += pd.date_range("14:00", "17:00", freq="1h").strftime("%H:%M").tolist()

        # Validar que haya suficiente espacio para la duración
        horas_inicio = [h for h in horas_validas if pd.to_datetime(h) + pd.Timedelta(hours=duracion) <= pd.to_datetime("22:00")]
        if not horas_inicio:
            continue

        hi = np.random.choice(horas_inicio)
        hf = (pd.to_datetime(hi) + pd.Timedelta(hours=duracion)).strftime("%H:%M")
        if 'dia' not in locals():
            dia = np.random.choice(["LUNES", "MARTES", "MIERCOLES", "JUEVES", "VIERNES", "SABADO"])

        filas_disponibilidad.append({
            "CodDocente": cod_docente,
            "CodAsignatura": cod_asig,
            "Día": dia,
            "HoraInicio": hi,
            "HoraFin": hf,
            "DuracionHoras": duracion,
            "Ciclo": ciclo
        })


In [3]:
# Convertir a DataFrame
df_result = pd.DataFrame(filas_disponibilidad)
df_result.shape
# Exportar
#df_result.to_csv("..\\Data\\Extraidos\\disponibilidad_docentes_simulada.csv", index=False, encoding="utf-8-sig")
#print(f"Total de docentes generados: {df_result['CodDocente'].nunique()}")
#print(df_result.head())

(71, 7)

In [4]:
import pandas as pd

sum_horario_programado = pd.read_csv("..\\Data\\Extraidos\\sum_horario_programado2025I.csv", sep=";", encoding="utf-8-sig")
sum_horario_programado.head()

,CodAsignatura,NomAsignatura,Créditos,CodDocente,NomDocente,Sección,Aula,Día,HoraInicio,HoraFin,Tope,Matriculados
0,INE002,PROGRAMACIÓN Y COMPUTACIÓN,2.0,0A0182,"RUIZ RIVERA, MARIA ELENA",2,211,VIERNES,14:00,15:00,4,2
1,INE002,PROGRAMACIÓN Y COMPUTACIÓN,2.0,0A0182,"RUIZ RIVERA, MARIA ELENA",2,211,VIERNES,15:00,17:00,4,2
2,INE002,PROGRAMACIÓN Y COMPUTACIÓN,2.0,07240690,"DAMASO RIOS, MARIA ROSA",3,211,MARTES,10:00,11:00,4,3
3,INE002,PROGRAMACIÓN Y COMPUTACIÓN,2.0,07240690,"DAMASO RIOS, MARIA ROSA",3,211,MARTES,11:00,13:00,4,3
4,INE002,PROGRAMACIÓN Y COMPUTACIÓN,2.0,085774,"VERA POMALAZA, VIRGINIA",5,211,MIERCOLES,10:00,11:00,4,1


In [5]:
sum_horario_programado.shape

(224, 12)

In [6]:
sum_horario_programado['HoraInicio'] = pd.to_datetime(sum_horario_programado['HoraInicio'], format='%H:%M')
sum_horario_programado['HoraFin'] = pd.to_datetime(sum_horario_programado['HoraFin'], format='%H:%M')

In [7]:
# Agrupamos por los campos comunes y tomamos el mínimo de inicio y máximo de fin
sum_horario_programado_unificado = sum_horario_programado.groupby(['CodAsignatura','CodDocente','Día'], as_index=False).agg({
    'HoraInicio': 'min',
    'HoraFin': 'max'
})

In [8]:
sum_horario_programado_unificado.head()

,CodAsignatura,CodDocente,Día,HoraInicio,HoraFin
0,202W0301,085774,LUNES,1900-01-01 14:00:00,1900-01-01 19:00:00
1,202W0301,0A1609,JUEVES,1900-01-01 17:00:00,1900-01-01 22:00:00
2,202W0301,0A9179,LUNES,1900-01-01 08:00:00,1900-01-01 13:00:00
3,202W0302,007056,JUEVES,1900-01-01 08:00:00,1900-01-01 13:00:00
4,202W0302,043788,JUEVES,1900-01-01 08:00:00,1900-01-01 13:00:00


In [9]:
sum_horario_programado_unificado.shape

(112, 5)

In [10]:
# Si deseas volver a tener las horas en formato string 'HH:MM'
sum_horario_programado_unificado['HoraInicio'] = sum_horario_programado_unificado['HoraInicio'].dt.strftime('%H:%M')
sum_horario_programado_unificado['HoraFin'] = sum_horario_programado_unificado['HoraFin'].dt.strftime('%H:%M')

In [11]:
duracion_asignatura = pd.read_csv("..\\Data\\Extraidos\\duracion_asignatura.csv", sep=";", encoding="utf-8-sig")
duracion_asignatura.head()

,NomAsignatura,DuracionHoras,Ciclo,CodAsignatura
0,ALGORÍTMICA I,5.0,3,202W0301
1,ESTADÍSTICA,5.0,3,202W0302
2,FISICA ELÉCTRONICA,4.0,3,202W0303
3,INGENIERÍA ECONÓMICA,4.0,3,202W0304
4,INTRODUCCIÓN AL DESARROLLO DE SOFTWARE,4.0,3,202W0305


In [12]:
resultado = pd.merge(
    sum_horario_programado_unificado,
    duracion_asignatura[["CodAsignatura", "DuracionHoras","Ciclo"]],
    on="CodAsignatura",
    how='left'
)

In [13]:
resultado.head()

,CodAsignatura,CodDocente,Día,HoraInicio,HoraFin,DuracionHoras,Ciclo
0,202W0301,085774,LUNES,14:00,19:00,5.0,3
1,202W0301,0A1609,JUEVES,17:00,22:00,5.0,3
2,202W0301,0A9179,LUNES,08:00,13:00,5.0,3
3,202W0302,007056,JUEVES,08:00,13:00,5.0,3
4,202W0302,043788,JUEVES,08:00,13:00,5.0,3


In [14]:
resultado_filtrado=resultado[["CodDocente","CodAsignatura","Día","HoraInicio","HoraFin","DuracionHoras","Ciclo"]]
resultado_filtrado.head()

,CodDocente,CodAsignatura,Día,HoraInicio,HoraFin,DuracionHoras,Ciclo
0,085774,202W0301,LUNES,14:00,19:00,5.0,3
1,0A1609,202W0301,JUEVES,17:00,22:00,5.0,3
2,0A9179,202W0301,LUNES,08:00,13:00,5.0,3
3,007056,202W0302,JUEVES,08:00,13:00,5.0,3
4,043788,202W0302,JUEVES,08:00,13:00,5.0,3


In [15]:

resultado_final = pd.concat([resultado_filtrado, df_result], ignore_index=True)
resultado_final

,CodDocente,CodAsignatura,Día,HoraInicio,HoraFin,DuracionHoras,Ciclo
0,085774,202W0301,LUNES,14:00,19:00,5.0,3
1,0A1609,202W0301,JUEVES,17:00,22:00,5.0,3
2,0A9179,202W0301,LUNES,08:00,13:00,5.0,3
3,007056,202W0302,JUEVES,08:00,13:00,5.0,3
4,043788,202W0302,JUEVES,08:00,13:00,5.0,3
...,...,...,...,...,...,...,...
178,Z000072,INO102,MIERCOLES,11:00,14:00,3.0,1
179,Z000073,INO103,MIERCOLES,12:00,15:00,3.0,1
180,Z000074,INO106,LUNES,10:00,16:00,6.0,1
181,Z000075,INO204,LUNES,11:00,17:00,6.0,2


In [16]:
resultado_final.to_csv("..\\Data\\Extraidos\\disponibilidad_docentes_simulada.csv", index=False, encoding="utf-8-sig")
print(f"Total de docentes generados: {resultado_final['CodDocente'].nunique()}")
print(resultado_final.head())

Total de docentes generados: 140
  CodDocente CodAsignatura     Día HoraInicio HoraFin  DuracionHoras  Ciclo
0     085774      202W0301   LUNES      14:00   19:00            5.0      3
1     0A1609      202W0301  JUEVES      17:00   22:00            5.0      3
2     0A9179      202W0301   LUNES      08:00   13:00            5.0      3
3     007056      202W0302  JUEVES      08:00   13:00            5.0      3
4     043788      202W0302  JUEVES      08:00   13:00            5.0      3
